In [1]:
import pandas as pd
import os

In [ ]:
# The file path to your data
file_path = '/itf-fi-ml/shared/users/ziyuzh/svm/data/biograd/filtered_human.txt'

# Define column names based on your data structure
column_names = [
    'BioGRID Interaction ID', 'Entrez Gene Interactor A', 'Entrez Gene Interactor B',
    'BioGRID ID Interactor A', 'BioGRID ID Interactor B', 'Systematic Name Interactor A',
    'Systematic Name Interactor B', 'Official Symbol Interactor A', 'Official Symbol Interactor B',
    'Synonyms Interactor A', 'Synonyms Interactor B', 'Experimental System',
    'Experimental System Type', 'Author', 'Publication Source', 'Organism ID Interactor A',
    'Organism ID Interactor B', 'Throughput', 'Score', 'Modification', 'Qualifications', 'Tags',
    'Source Database', 'SWISS-PROT Accessions Interactor A', 'TREMBL Accessions Interactor A',
    'REFSEQ Accessions Interactor A', 'SWISS-PROT Accessions Interactor B',
    'TREMBL Accessions Interactor B', 'REFSEQ Accessions Interactor B', 'Ontology Term IDs',
    'Ontology Term Names', 'Ontology Term Categories', 'Ontology Term Qualifier IDs',
    'Ontology Term Qualifier Names', 'Ontology Term Types', 'Organism Name Interactor A',
    'Organism Name Interactor B'
]

# Read the CSV file, using 'names' to specify column names
connection_all = pd.read_csv(file_path, sep='\t', names=column_names, header=0)
connection_df = connection_all[['Official Symbol Interactor A', 'Official Symbol Interactor B']]

/tmp/ipykernel_1234536/350929334.py:21: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  connection_all = pd.read_csv(file_path, sep='\t', names=column_names, header=0)


,BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,...,TREMBL Accessions Interactor B,REFSEQ Accessions Interactor B,Ontology Term IDs,Ontology Term Names,Ontology Term Categories,Ontology Term Qualifier IDs,Ontology Term Qualifier Names,Ontology Term Types,Organism Name Interactor A,Organism Name Interactor B
0,117,84665,88,124185,106603,-,-,MYPN,ACTN2,CMD1DD|CMH22|MYOP|RCM4,...,Q59FD9|F6THM6,NP_001094|NP_001265272|NP_001265273,-,-,-,-,-,-,Homo sapiens,Homo sapiens
1,183,90,2339,106605,108625,-,-,ACVR1,FNTA,ACTRI|ACVR1A|ACVRLK2|ALK2|FOP|SKR1|TSRI,...,-,NP_002018,-,-,-,-,-,-,Homo sapiens,Homo sapiens


In [10]:
connection_df = connection_all[['Official Symbol Interactor A', 'Official Symbol Interactor B']]

In [11]:
#### string gene_id map
import os
local_stringdb = os.path.join('/itf-fi-ml/shared/users/ziyuzh/svm/data/stringdb','2023')

ppidf = pd.read_csv(os.path.join(local_stringdb,'9606.protein.info.v12.0.txt'), sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
ppidf['preferred_name'] = ppidf['preferred_name'].str.upper()
stringId2name = ppidf.set_index('#string_protein_id')['preferred_name'].to_dict()
name2stringId = ppidf.set_index('preferred_name')['#string_protein_id'].to_dict()
ppidf = pd.read_csv(os.path.join(local_stringdb,'9606.protein.aliases.v12.0.txt'), sep='\t', header=0, usecols=['#string_protein_id', 'alias']).drop_duplicates(['alias'], keep='first')
ppidf['alias'] = ppidf['alias'].str.upper()
aliases2stringId = ppidf.set_index('alias')['#string_protein_id'].to_dict()

def string_convert(gene):
    if gene in name2stringId.keys():
        return name2stringId[gene]
    elif gene in aliases2stringId.keys():
        return aliases2stringId[gene]
    else:
        return None

In [13]:
connection_df['string_A'] = connection_df['Official Symbol Interactor A'].map(string_convert)
connection_df['string_B'] = connection_df['Official Symbol Interactor B'].map(string_convert)

/tmp/ipykernel_1234536/661777363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connection_df['string_A'] = connection_df['Official Symbol Interactor A'].map(string_convert)
/tmp/ipykernel_1234536/661777363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connection_df['string_B'] = connection_df['Official Symbol Interactor B'].map(string_convert)


In [14]:
len(connection_df),len(connection_df.dropna())

(1181472, 1142566)

In [15]:
connection_df = connection_df.dropna()

In [18]:
connection_df[['string_A', 'string_B']].to_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/biograd/biograd_full.txt', sep='\t', index=False, header=False)

In [2]:
connection_df = pd.read_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/biograd/biograd_full.txt', sep='\t')

In [5]:
connection_df = connection_df.drop_duplicates()

In [6]:
connection_df.to_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/biograd/biograd_full.txt', sep='\t', index=False, header=False)

In [7]:
### PPI features from node2vec
from pecanpy import pecanpy as node2vec

def Runnode2vec(filepath):
    n2v = node2vec.SparseOTF(p=1, q=1, workers=4, verbose=True)

    edge_list = n2v.read_edg(filepath, weighted=False, directed=False)
    emd = n2v.embed(dim=128, num_walks=10, walk_length=80, window_size=10, epochs=10)

    n2v_emd = pd.DataFrame(emd, n2v.nodes)

    n2v_emd.columns = ['network_' + str(col) for col in n2v_emd.columns]

    n2v_emd = n2v_emd.reset_index().rename(columns={"index":"string_id"})

    return n2v_emd

In [8]:
ppi_features = Runnode2vec('/itf-fi-ml/shared/users/ziyuzh/svm/data/biograd/biograd_full.txt')

new_columns = ['string_id'] + [f'feature_{i}' for i, col in enumerate(ppi_features.columns) if col != 'string_id']
# Reorder the DataFrame so that 'string_id' is the first column
df_combined = ppi_features[['string_id'] + [col for col in ppi_features.columns if col != 'string_id']]
df_combined.columns = new_columns
df_combined.to_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/biograd/biograd_full_emb.csv', index=False)

  0%|          | 0/178690 [00:00<?, ?it/s]

In [9]:
df_combined

,string_id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128
0,9606.ENSP00000405004,-0.602745,0.184485,-0.056290,0.200283,0.287977,0.128751,-0.266577,-0.447818,-0.219731,...,0.472205,-0.049370,-0.507523,-0.411684,0.157373,-0.321404,-0.280343,-0.128285,-0.227794,0.054549
1,9606.ENSP00000303423,-0.017925,-0.085430,0.324062,-0.015792,-0.237982,-0.011226,0.143389,-0.219723,-0.146273,...,0.271316,-0.009568,-0.019633,-0.083699,-0.242825,-0.262689,-0.108758,-0.087458,0.088219,0.134609
2,9606.ENSP00000345681,-0.247856,0.058089,0.389686,-0.107267,-0.287661,-0.144883,0.216122,-0.025241,0.108042,...,0.029950,-0.446196,-0.119484,0.150412,-0.210886,-0.203271,0.112209,0.355429,0.004097,-0.033894
3,9606.ENSP00000268058,0.007599,-0.097206,0.055421,0.042472,0.146485,0.007932,-0.014582,-0.104821,0.089242,...,0.047757,0.086790,0.243289,-0.098899,0.116015,0.060458,-0.004012,0.035345,-0.083233,-0.034158
4,9606.ENSP00000363017,-0.140522,-0.011645,0.024924,0.041777,0.171256,0.099754,0.002349,0.002255,-0.102872,...,0.092619,0.147346,0.114573,-0.021718,-0.095271,0.140402,-0.298429,0.043171,-0.062587,-0.190763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17864,9606.ENSP00000493312,-0.436426,-0.202854,-0.546597,-0.264880,0.187613,0.144841,-0.045859,0.580874,-0.138730,...,-0.055141,-0.187607,-0.056788,0.318108,-0.389103,0.314236,-0.141231,0.474819,0.132845,-0.047621
17865,9606.ENSP00000392252,-0.198048,0.306458,0.107102,-0.244255,0.522036,0.406114,-0.209284,-0.016561,-0.040061,...,0.161142,-0.433420,-0.054280,0.578865,-0.422933,0.374235,0.098368,0.037024,-0.107135,-0.053990
17866,9606.ENSP00000480601,0.150248,-0.031007,0.069398,-0.303524,-0.078832,0.060225,0.222359,-0.308579,0.109402,...,0.162197,0.079227,-0.036345,-0.182724,-0.143149,0.230563,0.019398,-0.101775,0.111779,-0.113535
17867,9606.ENSP00000319744,0.363844,-0.370964,-0.105327,0.354804,0.181508,0.489910,-0.507252,-0.136808,-0.700586,...,-0.187752,-0.259290,0.078485,-0.276690,-0.240438,-0.450400,-0.178799,0.550402,-0.294283,-0.183783


In [20]:
df_combined

,string_id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128
0,9606.ENSP00000351790,-0.013228,0.339514,-0.086623,-0.522640,-0.434300,0.294341,0.071298,-0.402676,0.291262,...,-0.023221,-0.141846,0.062596,-0.397049,0.155927,0.187512,0.036625,-0.271687,-0.492712,0.351162
1,9606.ENSP00000355537,0.069786,0.319310,-0.214684,-0.219110,-0.116415,-0.106649,0.057452,-0.234734,-0.042330,...,0.157409,0.177035,0.017093,0.025661,0.109434,0.360523,-0.074128,-0.414381,-0.240117,0.004755
2,9606.ENSP00000405004,0.109681,-0.409118,0.041274,0.715292,-0.025855,-0.246146,-0.173189,-0.213931,-0.178489,...,0.087959,0.185942,0.468800,0.155413,-0.035579,-0.429788,-0.117075,-0.341998,-0.178597,-0.054949
3,9606.ENSP00000303423,-0.130860,0.026562,-0.068850,0.221633,-0.284238,0.093171,-0.000626,-0.261085,-0.043036,...,-0.161839,0.117687,-0.168790,0.418245,-0.142504,-0.330475,-0.311156,0.015261,0.041238,0.266645
4,9606.ENSP00000345681,0.044662,0.126096,-0.066773,-0.120616,0.101672,0.059838,0.026608,-0.251712,-0.052006,...,-0.282942,0.115136,-0.303386,0.158665,0.056076,-0.227443,-0.289723,-0.054126,-0.032509,0.036373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17864,9606.ENSP00000493312,-0.102712,0.100174,0.169584,0.121295,-0.118873,-0.118564,0.001542,-0.415349,-0.117531,...,0.327147,0.005572,-0.205882,-0.011611,-0.125252,0.259344,0.073375,-0.408934,-0.279630,0.164069
17865,9606.ENSP00000392252,-0.016879,0.003898,-0.291863,0.102133,-0.692851,-0.633554,0.345543,-0.173736,0.126593,...,0.253676,-0.253478,-0.208466,0.024977,-0.096000,-0.144799,0.523497,-0.295949,-0.003834,-0.085689
17866,9606.ENSP00000480601,0.041860,0.135953,-0.087026,-0.508638,-0.221344,-0.212090,-0.135695,-0.388654,-0.214175,...,-0.069898,-0.343194,0.129187,-0.026626,0.032855,0.258506,-0.024004,-0.199813,-0.054386,0.311272
17867,9606.ENSP00000319744,-0.349995,-0.319127,-0.670831,0.179153,-0.302751,-0.126227,0.267126,-0.408250,0.239046,...,0.287804,0.005112,-0.548040,0.028053,-0.517071,-0.509400,0.027994,-0.247139,0.203089,0.447643
